In [1]:
import pandas as pd
import torch
from tqdm.auto import tqdm

In [14]:
df = pd.read_csv("data/SA_history_clicks_of_98.csv")
df

,queryText,packageName
0,توشمال,com.example.masood.yocheneapp
1,rush,com.ketchapp.rush
2,شب چله,yalda.ir.kr.si.ma.usn
3,عشق,com.roman.jadogareeshgh
4,کلابی,ir.mmdali.cluby
...,...,...
10948187,حروف الفبا,com.farad.entertainment.kidspersianalphabet
10948188,رویال,com.supercell.clashroyale
10948189,فرار از جنگل,com.mousecity.faraway2
10948190,روبینو,ir.rubx.bapp


In [15]:
def label_packages(packages: list):
    package_to_id = dict()
    id_to_package = dict()
    id_counter = 0
    for package in packages:
        if package not in package_to_id:
            package_to_id[package] = id_counter
            id_to_package[id_counter] = package
            id_counter += 1
    return package_to_id, id_to_package

package_to_id, id_to_package = label_packages(df["packageName"])
print(len(package_to_id))
len(id_to_package)


93592


93592

In [16]:
df[10:20]

,queryText,packageName
10,استعلام بیمه تامین اجتماعی,ir.sso.mobile
11,گاج,ir.gaj.gajmarket
12,دخل وخرج خانه,omd.secuso.org.dakhlokharg
13,پت ومت,com.tereza14.patomatbamaze
14,جوک,com.aseman.abi.apps.donya.jok
15,jevels,com.ogien.threejewled
16,س,ir.ali.s.masazh.darmani
17,نرمش همراه پایه,com.rahesabz.pocketstretch.free
18,radio javan,HosseinJavanAlmasi.Javan.Player
19,برنامه های جالب,lovefriend.screenroid.fingerprint


In [28]:
def count_unique_words(corpus):
    words_freq = dict()
    for sentence in tqdm(corpus):
        # print(sentence)
        for word in str(sentence).split():
            if word not in words_freq:
                words_freq[word] = 1
            else:
                words_freq[word] += 1
    return words_freq
    
words_freq = count_unique_words(df["queryText"])
len(words_freq)

590280

In [72]:
t = tokenizer("روبینو")
print(t)
tokenizer.decode(t["input_ids"])
len(tokenizer.get_vocab())

{'input_ids': [2, 21050, 1154, 4], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}


100000

'[CLS] radio javan [SEP]'

In [47]:
t["input_ids"]

[2, 65879, 80900, 1188, 4]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [ ]:
tokenizer.save_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer.save_vocabulary("HooshvareLab/bert-base-parsbert-uncased")

In [15]:
import random
from transformers import AutoTokenizer

class DatasetHandler:
    def __init__(self, file_address):
        self.df = pd.read_csv(file_address)
        self.df["similar"] = 1
        tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
        self.package_to_id, self.id_to_package = label_packages(df["packageName"], start_id=len(tokenizer.get_vocab()))
        
    def label_packages(packages: list, start_id=0):
        package_to_id = dict()
        id_to_package = dict()
        id_counter = start_id
        for package in tqdm(packages):
            if package not in package_to_id:
                package_to_id[package] = id_counter
                id_to_package[id_counter] = package
                id_counter += 1
        return package_to_id, id_to_package

    def exec_negative_sampling(self, neg_to_pos_ratio:int=5):
        negative_rows = []
        df_dict = dataset_handler.df.to_dict()
        for package_name_key in tqdm(df_dict["packageName"]):
            random_package = random.choice(self.package_to_id.keys())
            while random_package == df_dict["packageName"][package_name_key]:
                random_package = random.choice(self.package_to_id.keys())
            negative_rows.append({'queryText': row["queryText"], "packageName": row["packageName"], "similar": 0})
            neg_df = pd.DataFram(out)
        return neg_df

dataset_handler = DatasetHandler("data/SA_history_clicks_of_98.csv")

KeyboardInterrupt: 

In [10]:
df_dict = dataset_handler.df.to_dict()

In [11]:
df_dict

{'queryText': {0: 'توشمال',
  1: 'rush',
  2: 'شب چله',
  3: 'عشق',
  4: 'کلابی',
  5: 'دانلود از استوری',
  6: 'پاورپوینت',
  7: 'هکر وای فای قوی',
  8: 'روبینو',
  9: 'منج',
  10: 'استعلام بیمه تامین اجتماعی',
  11: 'گاج',
  12: 'دخل وخرج خانه',
  13: 'پت ومت',
  14: 'جوک',
  15: 'jevels',
  16: 'س',
  17: 'نرمش همراه پایه',
  18: 'radio javan',
  19: 'برنامه های جالب',
  20: 'گاوصندوق مخفی',
  21: 'fotmob',
  22: 'پخش توسط بلوث',
  23: 'بوبجی',
  24: 'کندی کراش',
  25: 'داییناسر',
  26: 'شوخی با بچه ها',
  27: 'کتاب بالابردن اعتمادبنفس',
  28: 'مخابرات من',
  29: 'فارماتون',
  30: 'رمان های طنز و کلکلی',
  31: 'فتوگرید',
  32: 'xnxx',
  33: 'برنامه مدل لباس ومانتو',
  34: 'برنامه بریدن تصاویر',
  35: 'روبینو',
  36: 'اناوالسا',
  37: 'lep s world 3',
  38: 'vulet',
  39: 'اکسل',
  40: 'دانلود پلنت زامبی 2',
  41: 'زپ',
  42: '1 حزحفح',
  43: 'صدا لمس',
  44: 'شترنج',
  45: 'ادیت تصویر',
  46: 'کتاب های گاج هشتم',
  47: 'security master',
  48: 'فیفا موبایل',
  49: 'twitch',
  50: 'ک

In [7]:
dataset_handler.exec_negative_sampling(1)

dataset_handler.df

KeyboardInterrupt: 